In [16]:
from google.colab import drive
drive.mount('/content/drive')
import os
import sys

os.chdir('/content/drive/MyDrive/Colab_Notebooks/Langchain')
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/Langchain')

Mounted at /content/drive


In [25]:
pip install langchain langchain-openai langchain_community pypdf sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.1 MB/s eta 0:00:00


In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Définir le modèle DeepSeek via OpenRouter
llm = ChatOpenAI(
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key="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",  # Remplace par ta clé OpenRouter
    model_name="deepseek/deepseek-r1:free"
)

# PromptTemplate avec un prompt système
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Tu es un assistant utile. Lorsque l'utilisateur donne une couleur, tu dois répondre "
     "avec une liste d'animaux qui ont cette couleur. Réponds uniquement avec une liste JSON "
     "comme ceci : animal : [\"animal1\", \"animal2\", ...]. Ne donne aucune explication."),
    ("human", "Quels animaux ont la couleur {couleur} ?")
])



# -----------------
def get_animals(couleur) :
  # Créer la chaîne (seul cette méthode marche avec LLMChain ça ne marche pas)
  chain = prompt | llm
  # Entrée utilisateur
  return chain.invoke({"couleur": couleur}).content


In [14]:
# Affichage du résultat
print("Résultat :")
print(get_animals("blanc"))

Résultat :
["polar bear", "swan", "dove", "sheep", "goat", "arctic fox", "white rabbit", "albino peacock", "beluga whale", "egret", "snowy owl", "white tiger", "albino alligator", "white-tailed deer", "white lion", "ermine", "white cockatoo", "samoyed dog", "white peacock", "seagull"]


# Mise en place du RAG

In [26]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


# --- Charger et découper le PDF ---
loader = PyPDFLoader("58210dec-e7b5-49a8-96b4-b5bc55f71c52.pdf")
pages = loader.load_and_split()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = splitter.split_documents(pages)

# 2. Embeddings locaux avec Hugging Face
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3. Indexation dans FAISS
vectorstore = FAISS.from_documents(documents, embedding)

In [29]:
# --- Faire une recherche par exemple ---
question = "Quelles sont les recommandations du rapport sur la formation ?"
retrieved_docs = vectorstore.similarity_search(question, k=4)

for doc in retrieved_docs :
  print( "Page ", doc.metadata['page']," : " ,  doc.page_content)

Page  73  :  Ce flou ne se lèvera que progressivement  Là où le soutien public sera utile, c’est pour appuyer 
la cartographie continue des besoins et des modalités de formations, pour apporter de la 
lisibilité dans une offre pléthorique, et pour assurer la formation des demandeurs d’emplois 
Recommandation n° 7
Investir dans la formation professionnelle continue des actifs et les dispositifs 
de formation autour de l’IA.
Page  108  :  encore la création d’une formation occupent au moins un tiers de l’emploi du temps d’un 
chercheur dans le public  Ce sont autant d’activités longues, laborieuses, décourageantes et à 
faible valeur ajoutée par rapport aux compétences des chercheurs   Elles ralentissent donc la 
production de nouvelles connaissances   Dans un monde de l’innovation en IA qui progresse 
chaque jour, ces complexités sont impensables
Page  115  :  d’affaires  Lieu de formation de grands chercheurs contemporains en IA, la 
France accentue l’effort de recherche et de formatio

In [37]:
# --- Charger les prompts ---
with open("prompt_system.txt", "r", encoding="utf-8") as f:
    system_prompt = f.read()

with open("prompt_context.txt", "r", encoding="utf-8") as f:
    user_template = f.read()


question = "Quelles sont les recommandations du rapport sur la formation ?"
retrieved_docs = vectorstore.similarity_search(question, k=4)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# --- Construire le prompt complet ---
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", user_template), # contient {context}
    ("human", "{question}")  # contient la vraie question dynamique
])

# --- Préparer le modèle ---
llm = ChatOpenAI(
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key="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",
    model_name="deepseek/deepseek-r1:free"
)

# --- Exécution de la chaîne ---
chain = prompt | llm
response = chain.invoke({"context": context,
                        "question": question })


In [38]:
# 9. --- Afficher la réponse finale ---
print("\n✅ Réponse générée :\n")
print(response.content)


✅ Réponse générée :

Les recommandations du rapport sur la formation sont les suivantes :

1. **Recommandation n°6**  
   *Objectif* : Généraliser l'IA dans l'enseignement supérieur et secondaire, et rendre accessibles les formations spécialisées.  
   *Pilotes* : Ministère de l'Éducation nationale et de l'Enseignement supérieur.  
   *Impact financier* : 1,2 milliard d’€ sur 5 ans.

2. **Recommandation n°7**  
   *Objectif* : Investir dans la **formation professionnelle continue** des travailleurs et les dispositifs liés à l'IA.  
   *Pilotes* : Ministère du Travail et de la Fonction publique.  
   *Impact financier* : 200 millions d’€ sur 5 ans.

3. **Recommandation n°8**  
   *Objectif* : Former les **professions créatives** à l'IA dès l'enseignement supérieur et en continu.  
   *Pilotes* : Ministère de la Culture et de l'Enseignement supérieur.  
   *Impact financier* : 20 millions d’€ sur 5 ans.

4. **Recommandation n°11**  
   *Objectif* : Expérimenter et évaluer les outils d'I